# Project 3

In [ ]:
import pokebase as pb
pb.cache.API_CACHE

import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

In [ ]:
fireTypePokemon = pb.APIResource('type', 'fire').pokemon
dfFire = {
    'name': [],
    'weight': [],
    'height': [],
    # 'moves': [],
    }
dfMoves = {
    'pokemon': [],
    'move': [],
}
for pokemon in fireTypePokemon[:10]:
    dfFire['name'].append(pokemon.pokemon.name)
    # display(pb.APIResource('pokemon', pokemon.pokemon.name).weight)
    dfFire['weight'].append(pokemon.pokemon.weight)
    dfFire['height'].append(pokemon.pokemon.height)
    for move in pokemon.pokemon.moves:
        dfMoves['pokemon'].append(pokemon.pokemon.name)
        dfMoves['move'].append(move.move.name)
    # display(pokemon.pokemon.name)
    break
# display(move for move in dfFire['moves'])
# for move in dfFire['moves'][0]:
#     display(move.move.name)
dfFire
display(pd.DataFrame(dfFire))
display(pd.DataFrame(dfMoves))
# pb.pokemon()

In [ ]:
gen1 = pb.generation('generation-1')
dir(gen1)
bulbasaur = gen1.results[0].pokemon_species[0]
bulbasaur = pb.pokemon(bulbasaur.name)
bulbasaur.moves[0].move.name
# dfFire = {
#     'name': [],
#     'weight': [],
#     'height': [],
#     # 'moves': [],
#     }
# dfMoves = {
#     'pokemon': [],
#     'move': [],
# }
# for pokemon in fireTypePokemon[:10]:
#     dfFire['name'].append(pokemon.pokemon.species)
#     # display(pb.APIResource('pokemon', pokemon.pokemon.name).weight)
#     dfFire['weight'].append(pokemon.pokemon.weight)
#     dfFire['height'].append(pokemon.pokemon.height)
#     for move in pokemon.pokemon.moves:
#         dfMoves['pokemon'].append(pokemon.pokemon.name)
#         dfMoves['move'].append(move.move.name)
#     # display(pokemon.pokemon.name)
#     break
# # display(move for move in dfFire['moves'])
# # for move in dfFire['moves'][0]:
# #     display(move.move.name)
# dfFire
# display(pd.DataFrame(dfFire))
# display(pd.DataFrame(dfMoves))
# # pb.pokemon()

In [ ]:
# gen1 = pb.generation(1).pokemon_species
# bulbasaur = gen1[0]
# bulbasaur.name
# fireTypePokemon = pb.APIResource('type', 'fire').pokemon
# dfFire = {
#     'species': [],
#     # 'weight': [],
#     # 'height': [],
#     # 'moves': [],
#     }
# # dfMoves = {
# #     'pokemon': [],
# #     'move': [],
# # }
# for pokemon in fireTypePokemon:
#     dfFire['species'].append(pokemon.pokemon.species.name)
#     break
# pd.DataFrame(dfFire)


## Gathering all Generation 1 Pokemon from [PokeAPI](https://pokeapi.co/)

In [ ]:
gen1 = requests.get('https://pokeapi.co/api/v2/generation/1')
gen1 = json.loads(gen1.content)

gen1Moves = gen1['moves']   # a list of moves included in the gen 1 games
gen1Types = gen1['types']   # a list of valid pokemon types in the gen 1
gen1Species = gen1['pokemon_species']   # a list of pokemon species introduced in gen 1

# Get pokemon objects for each of the pokemon from gen 1
gen1Pokemon = []
for species in gen1Species:
    pokemon = requests.get(f"https://pokeapi.co/api/v2/pokemon/{species['name']}")
    pokemon = json.loads(pokemon.content)
    gen1Pokemon.append(pokemon)

## Converting Pokemon JSON data into usable dataframes

In [ ]:
# Create a dataframe for moves included in the gen 1 games
# (to be used in validating moves listed for each pokemon)
movesDF = pd.DataFrame(gen1Moves)
movesDF = movesDF[['name']]
movesDF.columns = ['move']
movesDF

In [ ]:
# Create a dataframe of each pokemon and its various stats
pokemonDF = {
    'name': [],
    'base_experience': [],  # experience gained from defeating this pokemon
    'height': [],   # in decimetres
    'weight': [],   # in hectograms
    'abilities': [],    # total number of abilities
    # 'moves': [],    # total number of moves learnable by this pokemon
    'type': [],
    'type-slot': [],    # denotes whether the recorded type is the pokemon's primary or secondary type
    # All stat values represent base stats
    'hp': [],
    'attack': [],
    'defense': [],
    'special-attack': [],
    'special-defense': [],
    'speed': [],
}

pokemonMovesDF = {  # all gen 1 moves learnable by pokemon in gen 1
    'name': [],
    'move': [],
}

for pokemon in gen1Pokemon:
    typeSlot = 1
    for type in pokemon['types']:
        pokemonDF['type-slot'].append(type['slot'])
        typeSlot += 1
        pokemonDF['type'].append(type['type']['name'])

        pokemonDF['name'].append(pokemon['name'])
        pokemonDF['base_experience'].append(pokemon['base_experience'])
        pokemonDF['height'].append(pokemon['height'])
        pokemonDF['weight'].append(pokemon['weight'])
        pokemonDF['abilities'].append(len(pokemon['abilities']))

        # Add moves to a dictionary for now. 
        # Validity of these moves and a move count for each pokemon will be checked later
        for move in pokemon['moves']:
            pokemonMovesDF['name'].append(pokemon['name'])
            pokemonMovesDF['move'].append(move['move']['name'])

        for stat in pokemon['stats']:
            pokemonDF[stat['stat']['name']].append(stat['base_stat'])

# Create a dataframe from the moves gathered from the pokemon
pokemonMovesDF = pd.DataFrame(pokemonMovesDF)
# Purge the overall moves list of non-gen-1 moves
pokemonMovesDF = pd.merge(pokemonMovesDF, movesDF, how='right')

# Count the number of valid gen 1 moves learnable by each pokemon
pokemonMoveCountsDF = pokemonMovesDF.groupby('name')['move'].count().reset_index()
pokemonMoveCountsDF.columns = ['name', 'moves']

# Create a dataframe of the pokemon stats
pokemonDF = pd.DataFrame(pokemonDF)
# Utilize a gen 1 types dataframe to get rid of any invalid types
typesDF = pd.DataFrame(gen1Types)
typesDF = typesDF[['name']]
typesDF.columns = ['type']
pokemonDF = pd.merge(pokemonDF, typesDF, how='right')
# Finally, add the move counts of each pokemon
pokemonDF = pd.merge(pokemonDF, pokemonMoveCountsDF)
pokemonDF['stat-total'] = pokemonDF['hp'] + pokemonDF['attack'] + pokemonDF['defense'] + pokemonDF['special-attack'] + pokemonDF['special-defense'] + pokemonDF['speed'] 
pokemonDF

In [ ]:
pokemonDF.sort_values(by='stat-total')

### Pokemon Speeds by Type
Let's look at the Speed stat on the pokemon and see if there are any significant differences between the types.

In [ ]:
# display(pb.pokemon(1).types[0].type.name) 

# Load the dataset
pokemon_df = pd.read_csv('Pokemon.csv')

# Consolidate each Pokemon's types and speed
type1_df = pokemon_df[['Type1', 'Speed']].rename(columns={'Type1': 'Type'}) # Pokemon with one type
type2_df = pokemon_df[pokemon_df['Type2'].notnull() & (pokemon_df['Type2'] != ' ') & (pokemon_df['Type2'] != '0')][['Type2', 'Speed']].rename(columns={'Type2': 'Type'})
all_types_speed_df = pd.concat([type1_df, type2_df])
display(all_types_speed_df.head())
average_speed_by_type = all_types_speed_df.groupby('Type').mean().sort_values(by='Speed', ascending=False) # Calculate the average speed for each type

sns.set_style('whitegrid')

# Bar Chart for Average Speed of Pokémon by Type
plt.figure(figsize=(14, 8))
bar_chart = sns.barplot(x=average_speed_by_type.index, y="Speed", data=average_speed_by_type.reset_index())

for index, value in enumerate(average_speed_by_type['Speed']): # Add the average speed value on top of each bar
    bar_chart.text(index, value, round(value, 2), color='black', ha="center")
    
plt.title('Average Speed of Pokémon by Type')
plt.ylabel('Average Speed')
plt.xlabel('Type')
plt.show()

# Box Plot for Distribution of Speeds by Pokémon Type
plt.figure(figsize=(14, 8))
box_plot = sns.boxplot(x='Type', y='Speed', data=all_types_speed_df)
plt.title('Distribution of Speeds by Pokémon Type')
plt.ylabel('Speed')
plt.xlabel('Type')
plt.show()

Average weight for type normal: 632.9556962025316
Average weight for type fighting: 1209.2
Average weight for type flying: 666.6174496644295
Average weight for type poison: 926.6274509803922
Average weight for type ground: 1548.3870967741937
Average weight for type rock: 1200.0294117647059
Average weight for type bug: 643.0096153846154
Average weight for type ghost: 693.445652173913
Average weight for type steel: 2199.989010989011
Average weight for type fire: 1170.0
Average weight for type water: 921.5430107526881
Average weight for type grass: 621.296052631579
Average weight for type electric: 837.3
Average weight for type psychic: 971.4485294117648
Average weight for type ice: 1536.7121212121212
Average weight for type dragon: 2121.766355140187
Average weight for type dark: 989.7978723404256
Average weight for type fairy: 625.8795180722891

In [ ]:

# Check for missing values in the 'Speed' and 'Generation' columns
missing_values_check = pokemon_df[['Sp. Atk', 'Generation']].isnull().sum()

# Calculate the Pearson correlation coefficient
correlation_coefficient, p_value = pearsonr(pokemon_df['Sp. Atk'], pokemon_df['Generation'])

# Create a scatter plot with a regression line
plt.figure(figsize=(10, 6))
sns.regplot(x='Generation', y='Sp. Atk', data=pokemon_df, ci=None, scatter_kws={'alpha':0.5})
plt.title('Sp. Atk vs. Generation for Pokémon')
plt.xlabel('Generation')
plt.ylabel('Sp. Atk')
plt.show()

print(f"Pearson Correlation Coefficient: {correlation_coefficient}")
print(f"P-Value: {p_value}")
